In [51]:
import random
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.models as models

# Assign seed for repeatability
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define Residual Block

There are two types of residual blocks:
1. Basic Block. Used for ResNet18 and ResNet34
2. Bottleneck Block. Used for ResNet50, ResNet101, ResNet152

For now, we will implement the Basic Block. 

In a ResNet there are two types of shortcuts/identity mappings/skip connections:
- input and output of the same dimensions are added together
- input and output of the different dimensions are added together. In this case we perform downsampling.

In [52]:
class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super().__init__()
    # Declare the elements of a basic block
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                           stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.act = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                           stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.downsample = downsample

  def forward(self, x):

    identity_map = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.act(out)
    out = self.conv2(out)
    out = self.bn2(out)
    if self.downsample is not None:
      identity_map = self.downsample(x)
    # Add the identity map to the output of the convolution
    out += identity_map
    out = self.act(out)

    return out

In [53]:
# With no downsample
basic_block = BasicBlock(3, 64)
basic_block

BasicBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

# Define ResNet Layers

ResNet34 has 4 layers, each layer comprises of a sequence of blocks (that we defined earlier).

- The first layer has 3 blocks.
- The second layer has 4 blocks
- The third layer has 6 blocks.
- The final layer has 3 blocks.

In [54]:
class ResNetLayer(nn.Module):
  def __init__(self, block, in_channels, out_channels, n_blocks, stride=1):
    super().__init__()
    downsample = None
    if stride != 1 or in_channels != out_channels:
      downsample = nn.Sequential(nn.Conv2d(in_channels, out_channels, 
                                           kernel_size=1, stride=stride,
                                           bias=False),
                                 nn.BatchNorm2d(out_channels))
    layers = []
    layers.append(block(in_channels, out_channels, stride, downsample))
    for _ in range(1, n_blocks):
      layers.append(block(out_channels, out_channels))
    self.blocks = nn.Sequential(*layers)

  def forward(self, x):

    x = self.blocks(x)
    
    return x

In [55]:
resnet_layer1 = ResNetLayer(BasicBlock, 64, 64, 3)
resnet_layer1

ResNetLayer(
  (blocks): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),

In [56]:
resnet_layer2 = ResNetLayer(BasicBlock, 64, 128, 4, stride=2)
resnet_layer2

ResNetLayer(
  (blocks): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=

# Define ResNet model

We have defined the key elements of the ResNet model. Now, we will piece together the different elements to define the ResNet model architecture.


In [57]:
class ResNet(nn.Module):
  def __init__(self, block, layers, n_filters, n_classes):
    super().__init__()
    # Define the convolution block
    self.conv1 = nn.Conv2d(3, n_filters, kernel_size=7, stride=2, padding=3, 
                           bias=False)
    self.bn1 = nn.BatchNorm2d(n_filters)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    # Define the layers
    self.layer1 = ResNetLayer(block, n_filters, 64, layers[0])
    self.layer2 = ResNetLayer(block, 64, 128, layers[1], stride=2)
    self.layer3 = ResNetLayer(block, 128, 256, layers[2], stride=2)
    self.layer4 = ResNetLayer(block, 256, 512, layers[3], stride=2)

    # Average of activations across a convolutional grid
    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    # Final fully connected layer
    self.fc = nn.Linear(512, n_classes)

  def forward(self, x):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avg_pool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    return x

In [58]:
def resnet34(n_filters, n_classes):

  # Initialize seed for repeatability
  torch.manual_seed(SEED)

  # Number of layers in the ResNet34 model
  layers=[3, 4, 6, 3]
  model = ResNet(BasicBlock, layers, n_filters, n_classes)
  
  return model

In [59]:
# Initiate ResNet model
model = resnet34(64, 1000)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): ResNetLayer(
    (blocks): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

# Check if our model matches the model available via PyTorch

In [64]:
print("Number of parameters in our ResNet model:", sum(np.prod(parameters.shape)
  for parameters in model.parameters()))
print("Number of parameters in PyTorch ResNet model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet34(False).parameters()))

Number of parameters in our ResNet model: 21797672
Number of parameters in PyTorch ResNet model: 21797672


In [60]:
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [61]:
summary(models.resnet34(False), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

# References

[1] [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)

[2] [Residual Networks: Implementing ResNet in Pytorch](https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278)

[3] [Understanding and Building Resnet from scratch using Pytorch.](https://jarvislabs.ai/blogs/resnet)

[4] [Building ResNet34 in PyTorch](https://github.com/jarvislabsai/blog/blob/master/build_resnet34_pytorch/Building%20Resnet%20in%20PyTorch.ipynb)